In [15]:
import pandas as pd
from folium import plugins
import folium
from folium.plugins import HeatMap
import eeweather as ee

#http://eeweather.openee.io/en/latest/advanced.html#zcta-to-latitude-longitude-conversion
#https://alcidanalytics.com/p/geographic-heatmap-in-python
#https://alysivji.github.io/getting-started-with-folium.html

In [16]:
file = '/Users/gta/Desktop/average_test/test_data.csv'

In [17]:
df = pd.read_csv(file)

In [18]:
df

,SVC_ZIP,MTH_THM
0,91755,5000.000
1,91755,4500.000
2,91755,5000.000
3,91755,5000.000
4,92602,54.860
5,92602,85.436
6,91030,41.106
7,93010,42.780
8,93101,22.106
9,91745,72.500


In [19]:
#df['TOT_RECORDS'] = df.groupby('SVC_ZIP')['MTH_THM'].transform('count')
#df['ZIP_TOTAL'] = df.groupby('SVC_ZIP')['MTH_THM'].transform('sum')
#df['ZIP_TOTAL'] = df.groupby('SVC_ZIP')['MTH_THM'].transform('median')

In [20]:
count_ = df.groupby('SVC_ZIP', as_index=False).count()
count_

,SVC_ZIP,MTH_THM
0,91030,1
1,91745,3
2,91755,4
3,92602,2
4,93010,1
5,93101,6
6,94122,6


In [21]:
sum_ = df.groupby('SVC_ZIP', as_index=False).median()
sum_

,SVC_ZIP,MTH_THM
0,91030,41.106
1,91745,80.198
2,91755,5000.000
3,92602,70.148
4,93010,42.780
5,93101,92.500
6,94122,55.239


In [22]:
new_sum = sum_.rename(columns={'MTH_THM': 'THM_MEDIAN'})
new_count = count_.rename(columns={'MTH_THM': 'ZIP_CNT'})

In [23]:
new_df = pd.merge(new_count, new_sum, how='inner', on='SVC_ZIP')
new_df

,SVC_ZIP,ZIP_CNT,THM_MEDIAN
0,91030,1,41.106
1,91745,3,80.198
2,91755,4,5000.000
3,92602,2,70.148
4,93010,1,42.780
5,93101,6,92.500
6,94122,6,55.239


In [24]:
df_final = new_df.sort_values(['THM_MEDIAN'], ascending=False).reset_index(drop=True)

In [25]:
df_final.head(10)

,SVC_ZIP,ZIP_CNT,THM_MEDIAN
0,91755,4,5000.000
1,93101,6,92.500
2,91745,3,80.198
3,92602,2,70.148
4,94122,6,55.239
5,93010,1,42.780
6,91030,1,41.106


In [26]:
df_final['LAT'] = ''
df_final['LNG'] = ''

for index, row in df_final.iterrows():
    try:
        df_final.set_value(index, 'LAT', ee.zcta_to_lat_long(row['SVC_ZIP'])[0])
        df_final.set_value(index, 'LNG', ee.zcta_to_lat_long(row['SVC_ZIP'])[1])
    except:
        pass

/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/gta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [27]:
df_final

,SVC_ZIP,ZIP_CNT,THM_MEDIAN,LAT,LNG
0,91755,4,5000.000,34.0481,-118.115
1,93101,6,92.500,34.419,-119.709
2,91745,3,80.198,33.9993,-117.973
3,92602,2,70.148,33.7459,-117.749
4,94122,6,55.239,37.7588,-122.485
5,93010,1,42.780,34.2298,-119.075
6,91030,1,41.106,34.1102,-118.157


In [28]:
m = folium.Map([34.052235, -118.243683], zoom_start=8)

for index, row in df_final.iterrows():
    folium.CircleMarker([row['LAT'], row['LNG'], row['THM_MEDIAN']],
                        radius=0.5,
                        fill_color="#3db7e4",
                       ).add_to(m)

stationArr = df_final[['LAT', 'LNG', 'THM_MEDIAN']].values

m.add_child(plugins.HeatMap(stationArr, radius=15))
m

In [34]:
coordinates = [
    [42.3581, -71.0636],
    [42.82995815, -74.78991444],
    [43.17929819, -78.56603306],
    [43.40320216, -82.37774519],
    [43.49975489, -86.20965845],
    [41.4338549, -108.74485069],
    [40.67471747, -112.29609954],
    [39.8093434, -115.76190821],
    [38.84352776, -119.13665678],
    [37.7833, -122.4167]]

m = folium.Map(location=[41.9, -97.3], zoom_start=4)
my_PolyLine = folium.PolyLine(locations=coordinates,weight=1)
m.add_child(my_PolyLine)